# M7 Quickstart: Measurements and CLI
This notebook demonstrates the new measurement helpers and CLI schema/ordering introduced in M7.

In [ ]:
from pathlib import Path

from spicelab.analysis.measure import (
    ENOBSpec,
    GainBandwidthSpec,
    GainMarginSpec,
    PhaseMarginSpec,
    RiseTimeSpec,
    THDSpec,
    measure,
)
from spicelab.io import load_dataset

ac_raw = Path("tests/fixtures/rc_ac_ng.raw")
tran_raw = Path("tests/fixtures/rc_tran_ng.raw")
ds_ac = load_dataset(ac_raw, allow_binary=True)
ds_tr = load_dataset(tran_raw, allow_binary=True)
rows_ac = measure(
    ds_ac,
    [
        PhaseMarginSpec(name="pm", numerator="V(out)", denominator="I(R1)"),
        GainBandwidthSpec(name="gbw", numerator="V(out)", denominator="I(R1)"),
        GainMarginSpec(name="gm", numerator="V(out)", denominator="I(R1)"),
    ],
    return_as="python",
)
rows_tr = measure(
    ds_tr,
    [
        RiseTimeSpec(name="tr", signal="V(out)"),
        THDSpec(name="thd", signal="V(out)", harmonics=5, f0=None),
        ENOBSpec(name="enob", signal="V(out)", harmonics=5, f0=None),
    ],
    return_as="python",
)
rows_ac, rows_tr[:2]  # show a preview

## CLI schema and listing
You can inspect available signals and print a schema-only CSV header without executing any measurement.

In [ ]:
import subprocess
import sys

schema = subprocess.check_output(
    [
        sys.executable,
        "-m",
        "spicelab.cli.measure",
        str(ac_raw),
        "--ac",
        "--num",
        "V(out)",
        "--den",
        "I(R1)",
        "--format",
        "schema",
    ]
).decode()
signals = (
    subprocess.check_output(
        [
            sys.executable,
            "-m",
            "spicelab.cli.measure",
            str(ac_raw),
            "--list-signals",
            "--list-details",
            "--format",
            "csv",
        ]
    )
    .decode()
    .splitlines()[:5]
)
schema, signals